In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import train_test_split
#import tensorflow as tf
from sklearn.metrics import roc_auc_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae

In [2]:
df = pd.read_csv("/home/lizard/Documents/Indoor localization Dataset/TrainingData.csv")
from sklearn.utils import shuffle
df = shuffle(df,random_state=50)
df.tail()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
14565,100,100,100,100,100,100,100,100,100,100,...,100,-7376.631800,4.864855e+06,1,2,104,2,15,11,1371721935
15649,100,100,100,100,100,100,100,100,100,100,...,100,-7336.299000,4.864829e+06,1,2,111,2,16,14,1371720701
10123,100,100,100,100,100,100,100,100,100,100,...,100,-7336.299000,4.864829e+06,3,2,128,2,11,13,1371716769
5600,100,100,100,100,100,100,100,100,100,100,...,100,-7411.353136,4.864803e+06,0,2,201,1,9,14,1371720021
14000,100,100,100,100,100,100,100,100,100,100,...,100,-7331.918400,4.864823e+06,3,2,122,2,14,7,1371723396


In [3]:
X = df.drop([	'RELATIVEPOSITION',	'USERID',	'PHONEID',	'TIMESTAMP'],axis=1)
y = df['RELATIVEPOSITION']

In [4]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X)
scaled_data_X=scaler.transform(X)

In [5]:
X_new = SelectKBest(mutual_info_classif, k=300).fit_transform(scaled_data_X, y)
x_new_df = pd.DataFrame(X_new)

In [6]:
from sklearn.model_selection import RandomizedSearchCV

In [8]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    z= model.predict(X_test)
    p = precision_score(y_test,z, average='macro')
    m_score_test =model.score(X_test, y_test)
    m_score_train =model.score(X_train, y_train)
    r= recall_score(y_test,z,average='macro')
    f = f1_score(y_test,z, average='macro')
    confusion=confusion_matrix(y_test,z) 
    specificity = confusion[0,0]/(confusion[0,0]+confusion[0,1])
    gmean=np.sqrt(specificity*r)
    ROC_score=roc_auc_score(y_test,z, average=None)


    return m_score_test  ,m_score_train , p, r, f, specificity , gmean, ROC_score 

In [13]:
# THis is the beginning of Random Forest
from sklearn.ensemble import RandomForestClassifier

In [18]:
parameters = {'n_estimators': (20,40,60,80,100,120,150),'min_samples_split' : (2,4,8,10,15,20),'max_depth':(5,10,15,20,40)}
                                                                                                                                     
DT_grid_RF  = RandomizedSearchCV( RandomForestClassifier(), param_distributions = parameters, cv = 3, verbose = 10)
result_RF=DT_grid_RF.fit(x_new_df,y)
  
RF= result_RF.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START max_depth=40, min_samples_split=20, n_estimators=20........
[CV 1/3; 1/10] END max_depth=40, min_samples_split=20, n_estimators=20;, score=0.942 total time=   0.7s
[CV 2/3; 1/10] START max_depth=40, min_samples_split=20, n_estimators=20........
[CV 2/3; 1/10] END max_depth=40, min_samples_split=20, n_estimators=20;, score=0.942 total time=   0.8s
[CV 3/3; 1/10] START max_depth=40, min_samples_split=20, n_estimators=20........
[CV 3/3; 1/10] END max_depth=40, min_samples_split=20, n_estimators=20;, score=0.940 total time=   0.7s
[CV 1/3; 2/10] START max_depth=20, min_samples_split=8, n_estimators=60.........
[CV 1/3; 2/10] END max_depth=20, min_samples_split=8, n_estimators=60;, score=0.923 total time=   1.7s
[CV 2/3; 2/10] START max_depth=20, min_samples_split=8, n_estimators=60.........
[CV 2/3; 2/10] END max_depth=20, min_samples_split=8, n_estimators=60;, score=0.923 total time=   1.7s
[CV 3/3; 2/10] S

In [19]:
print(RF)

RandomForestClassifier(max_depth=40, min_samples_split=4, n_estimators=60)


In [14]:
scores_RF_test = []
scores_RF_train = []
precision_RF =[]
recall_RF=[]
f1_score_RF = []
specificity_RF =[]
gmean_RF= []
ROC_RF= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_RF, X_test_RF, y_train_RF, y_test_RF = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6,something8= get_score(RandomForestClassifier(max_depth=40, min_samples_split=4, n_estimators=60), X_train_RF, X_test_RF, y_train_RF, y_test_RF)
    scores_RF_test.append(something1)
    scores_RF_train.append(something7)
    precision_RF.append(something2)
    recall_RF.append(something3)
    f1_score_RF.append(something4)
    specificity_RF.append(something5)
    gmean_RF.append(something6)
    ROC_RF.append(something8)
print('Score test is :', scores_RF_test)
print('mean Score test is :', np.mean(scores_RF_test))
print('Score train is :', scores_RF_train)
print('mean Score train is :', np.mean(scores_RF_train))
print('Precision is :',precision_RF)
print('mean Precision is :',np.mean(precision_RF))
print('F1 score is :',f1_score_RF)
print('mean F1 score is :',np.mean(f1_score_RF))
print('REcall is :',recall_RF)
print('mean Recall is :',np.mean(recall_RF))
print('Gmean is :',gmean_RF)
print('mean Gmean is :',np.mean(gmean_RF))
print('ROC score is :', ROC_RF)
print('mean ROC score is :', np.mean(ROC_RF))

Score test is : [0.9719157472417251, 0.9719157472417251, 0.970912738214644, 0.9694082246740221, 0.9638916750250752, 0.966900702106319, 0.9699097291875627, 0.9703963873557451, 0.9719016557952835, 0.9749121926743602]
mean Score test is : 0.9702064799516462
Score train is : [0.9983837708298501, 0.99793791450705, 0.99849523491055, 0.9979936465474001, 0.9985509669509001, 0.9981051106281001, 0.99849523491055, 0.998773963441819, 0.9980494872938029, 0.9981052162282658]
mean Score train is : 0.9982890546248289
Precision is : [0.9836924868957484, 0.982172899881919, 0.9815962271529765, 0.9807333220166355, 0.9728549108345454, 0.9746935321804432, 0.9765369697498509, 0.9812984815105552, 0.9836829836829837, 0.9824437425491908]
mean Precision is : 0.9799705556454847
F1 score is : [0.9458094322957614, 0.9459545271505126, 0.9438740548777529, 0.9407318833270719, 0.9295673213721538, 0.9359603706010589, 0.9422485417322348, 0.942824370208398, 0.9458045217931785, 0.9521241227159868]
mean F1 score is : 0.9424

In [15]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

In [13]:
parameters = {'tol': (.0001,.001,.01,.0005,.05,.005),'C':(1,5,15,.05,.001)}
                                                                                                                                     
DT_grid_Lr  = RandomizedSearchCV( LogisticRegression(max_iter=1000), param_distributions = parameters, cv = 3, verbose = 10)
result_Lr=DT_grid_Lr.fit(x_new_df,y)
  
Lr= result_Lr.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START C=15, tol=0.0005...........................................
[CV 1/3; 1/10] END ............C=15, tol=0.0005;, score=0.855 total time=   6.9s
[CV 2/3; 1/10] START C=15, tol=0.0005...........................................
[CV 2/3; 1/10] END ............C=15, tol=0.0005;, score=0.861 total time=   8.6s
[CV 3/3; 1/10] START C=15, tol=0.0005...........................................
[CV 3/3; 1/10] END ............C=15, tol=0.0005;, score=0.861 total time=   7.5s
[CV 1/3; 2/10] START C=1, tol=0.005.............................................
[CV 1/3; 2/10] END ..............C=1, tol=0.005;, score=0.856 total time=   4.1s
[CV 2/3; 2/10] START C=1, tol=0.005.............................................
[CV 2/3; 2/10] END ..............C=1, tol=0.005;, score=0.861 total time=   6.1s
[CV 3/3; 2/10] START C=1, tol=0.005.............................................
[CV 3/3; 2/10] END ..............C=1, tol=0.005;

In [14]:
print(Lr)

LogisticRegression(C=1, max_iter=1000, tol=0.005)


In [17]:
scores_lr_test = []
scores_lr_train = []
precision_lr =[]
recall_lr=[]
f1_score_lr = []
specificity_lr =[]
gmean_lr= []
ROC_lr= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_lr, X_test_lr, y_train_lr, y_test_lr = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6,something8= get_score(LogisticRegression(C=1, max_iter=1000, tol=0.005), X_train_lr, X_test_lr, y_train_lr, y_test_lr)
    scores_lr_test.append(something1)
    scores_lr_train.append(something7)
    precision_lr.append(something2)
    recall_lr.append(something3)
    f1_score_lr.append(something4)
    specificity_lr.append(something5)
    gmean_lr.append(something6)
    ROC_lr.append(something8)
print('Score test is :', scores_lr_test)
print('mean Score test is :', np.mean(scores_lr_test))
print('Score train is :', scores_lr_train)
print('mean Score train is :', np.mean(scores_lr_train))
print('Precision is :',precision_lr)
print('mean Precision is :',np.mean(precision_lr))
print('F1 score is :',f1_score_lr)
print('mean F1 score is :',np.mean(f1_score_lr))
print('REcall is :',recall_lr)
print('mean Recall is :',np.mean(recall_lr))
print('Gmean is :',gmean_lr)
print('mean Gmean is :',np.mean(gmean_lr))
print('ROC score is :', ROC_lr)
print('mean ROC score is :', np.mean(ROC_lr))


Score test is : [0.854062186559679, 0.8565697091273822, 0.8655967903711134, 0.858074222668004, 0.8605817452357071, 0.8590772316950852, 0.8595787362086259, 0.868038133467135, 0.8660311088810838, 0.8665328650275966]
mean Score test is : 0.8614142729241413
Score train is : [0.8677478682494566, 0.8685838488547065, 0.8693640974196065, 0.8669676196845566, 0.8689739731371565, 0.8670790837652567, 0.8683051886529566, 0.8650245207311636, 0.8674765938475256, 0.8673651359786001]
mean Score train is : 0.8676887930320986
Precision is : [0.7517689632864306, 0.7561203172857929, 0.7810948876477386, 0.7675601964230909, 0.7686483939370141, 0.7630116814334504, 0.7675804490556861, 0.787362169817496, 0.7845699074503978, 0.7848453895440617]
mean Precision is : 0.771256235588116
F1 score is : [0.6644058680130824, 0.6774119829848855, 0.7017156074732969, 0.6701938210800514, 0.6864354240202734, 0.6846405256347696, 0.6809548952783244, 0.7086990857370873, 0.7003302876289966, 0.7009533157336583]
mean F1 score is : 

In [18]:
from sklearn.naive_bayes import GaussianNB

In [18]:
parameters = {'var_smoothing' : (2,4,6,10,15,20,60,80,40)}
                                                                                                                                     
DT_grid_Gnb  = RandomizedSearchCV( GaussianNB(), param_distributions = parameters, cv = 3, verbose = 10)
result_Gnb=DT_grid_Gnb.fit(x_new_df,y)
  
Gnb= result_Gnb.best_estimator_

/home/lizard/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START var_smoothing=2.............................................
[CV 1/3; 1/9] END ..............var_smoothing=2;, score=0.788 total time=   0.1s
[CV 2/3; 1/9] START var_smoothing=2.............................................
[CV 2/3; 1/9] END ..............var_smoothing=2;, score=0.791 total time=   0.1s
[CV 3/3; 1/9] START var_smoothing=2.............................................
[CV 3/3; 1/9] END ..............var_smoothing=2;, score=0.776 total time=   0.1s
[CV 1/3; 2/9] START var_smoothing=4.............................................
[CV 1/3; 2/9] END ..............var_smoothing=4;, score=0.824 total time=   0.1s
[CV 2/3; 2/9] START var_smoothing=4.............................................
[CV 2/3; 2/9] END ..............var_smoothing=4;, score=0.820 total time=   0.1s
[CV 3/3; 2/9] START var_smoothing=4.............................................
[CV 3/3; 2/9] END ..............var_smoothing=4;,

In [19]:
print(Gnb)

GaussianNB(var_smoothing=15)


In [19]:
# this is Naive bayes
scores_gb_test = []
scores_gb_train = []
precision_gb =[]
recall_gb=[]
f1_score_gb = []
specificity_gb =[]
gmean_gb= []
ROC_gb= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_gb, X_test_gb, y_train_gb, y_test_gb = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6,something8= get_score(GaussianNB(var_smoothing=15), X_train_gb, X_test_gb, y_train_gb, y_test_gb)
    scores_gb_test.append(something1)
    scores_gb_train.append(something7)
    precision_gb.append(something2)
    recall_gb.append(something3)
    f1_score_gb.append(something4)
    specificity_gb.append(something5)
    gmean_gb.append(something6)
    ROC_gb.append(something8)
print('Score test is :', scores_gb_test)
print('mean Score test is :', np.mean(scores_gb_test))
print('Score train is :', scores_gb_train)
print('mean Score train is :', np.mean(scores_gb_train))
print('Precision is :',precision_gb)
print('mean Precision is :',np.mean(precision_gb))
print('F1 score is :',f1_score_gb)
print('mean F1 score is :',np.mean(f1_score_gb))
print('REcall is :',recall_gb)
print('mean Recall is :',np.mean(recall_gb))
print('Gmean is :',gmean_gb)
print('mean Gmean is :',np.mean(gmean_gb))
print('ROC score is :', ROC_gb)
print('mean ROC score is :', np.mean(ROC_gb))

Score test is : [0.8340020060180542, 0.8360080240722166, 0.8289869608826479, 0.8340020060180542, 0.8309929789368105, 0.8335005015045135, 0.8340020060180542, 0.8354239839438033, 0.8253888610135475, 0.831409934771701]
mean Score test is : 0.8323717263179404
Score train is : [0.8331940032324583, 0.8332497352728083, 0.8348102324026082, 0.8299615448921585, 0.8312991138605584, 0.8336955915956084, 0.8335283954745584, 0.8335934016941596, 0.8342064199732501, 0.8335934016941596]
mean Score train is : 0.8331131840092327
Precision is : [0.6799413226962654, 0.6929924242424242, 0.6408587954231107, 0.6799488785396928, 0.6609220328604959, 0.6767665033913509, 0.6799699582027168, 0.689803976032697, 0.6124321736354932, 0.6573720397249809]
mean Precision is : 0.6671008104749229
F1 score is : [0.5451676814036104, 0.5583819570260249, 0.525175435104898, 0.5510557787758639, 0.5468166536844271, 0.5525872579472249, 0.5529799014544777, 0.5560547861850148, 0.5143190870741668, 0.5331511433351924]
mean F1 score is 

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
parameters_knn = {'n_neighbors': (1,2,3,4,5,10,15),'leaf_size':(15,30,50,100),'p':(1,2,3)}
knn_grid  = RandomizedSearchCV( KNeighborsClassifier(), param_distributions = parameters_knn, cv = 3, verbose = 10,scoring='accuracy')
result_knn=knn_grid.fit(x_new_df,y)
knn_estimator_result = result_knn.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START leaf_size=50, n_neighbors=5, p=3...........................
[CV 1/3; 1/10] END leaf_size=50, n_neighbors=5, p=3;, score=0.901 total time= 2.4min
[CV 2/3; 1/10] START leaf_size=50, n_neighbors=5, p=3...........................
[CV 2/3; 1/10] END leaf_size=50, n_neighbors=5, p=3;, score=0.905 total time= 2.3min
[CV 3/3; 1/10] START leaf_size=50, n_neighbors=5, p=3...........................
[CV 3/3; 1/10] END leaf_size=50, n_neighbors=5, p=3;, score=0.913 total time= 2.3min
[CV 1/3; 2/10] START leaf_size=30, n_neighbors=4, p=3...........................
[CV 1/3; 2/10] END leaf_size=30, n_neighbors=4, p=3;, score=0.902 total time= 2.3min
[CV 2/3; 2/10] START leaf_size=30, n_neighbors=4, p=3...........................
[CV 2/3; 2/10] END leaf_size=30, n_neighbors=4, p=3;, score=0.905 total time= 2.4min
[CV 3/3; 2/10] START leaf_size=30, n_neighbors=4, p=3...........................
[CV 3/3; 2/10] END leaf_size

In [24]:
print(knn_estimator_result)

KNeighborsClassifier(leaf_size=50, p=1)


In [21]:
scores_knn_test = []
scores_knn_train = []
precision_knn =[]
recall_knn=[]
f1_score_knn = []
specificity_knn =[]
gmean_knn= []
ROC_knn= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_knn, X_test_knn, y_train_knn, y_test_knn = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6,something8= get_score(KNeighborsClassifier(leaf_size=50, p=1), X_train_knn, X_test_knn, y_train_knn, y_test_knn)
    scores_knn_test.append(something1)
    scores_knn_train.append(something7)
    precision_knn.append(something2)
    recall_knn.append(something3)
    f1_score_knn.append(something4)
    specificity_knn.append(something5)
    gmean_knn.append(something6)
    ROC_knn.append(something8)
print('Score test is :', scores_knn_test)
print('mean Score test is :', np.mean(scores_knn_test))
print('Score train is :', scores_knn_train)
print('mean Score train is :', np.mean(scores_knn_train))
print('Precision is :',precision_knn)
print('mean Precision is :',np.mean(precision_knn))
print('F1 score is :',f1_score_knn)
print('mean F1 score is :',np.mean(f1_score_knn))
print('REcall is :',recall_knn)
print('mean Recall is :',np.mean(recall_knn))
print('Gmean is :',gmean_knn)
print('mean Gmean is :',np.mean(gmean_knn))
print('ROC score is :', ROC_knn)
print('mean ROC score is :', np.mean(ROC_knn))

Score test is : [0.9393179538615848, 0.9353059177532598, 0.9338014042126379, 0.9448345035105316, 0.9438314944834504, 0.9453360080240722, 0.9473420260782347, 0.9387857501254391, 0.9453085800301053, 0.9528349222277973]
mean Score test is : 0.9426698560307113
Score train is : [0.9640528339742518, 0.9647216184584517, 0.9639413698935518, 0.9636627096918018, 0.9621579446023519, 0.9632725854093518, 0.9637741737725019, 0.9630517164511815, 0.9633303611234953, 0.9634418189924209]
mean Score train is : 0.9635407132369359
Precision is : [0.9097018692197474, 0.8952335102468962, 0.8918912589070853, 0.9270934738409071, 0.9183025702949801, 0.9187114845938376, 0.9167247467707462, 0.9033989725689331, 0.9137842385715622, 0.9276656617120065]
mean Precision is : 0.9122507786726702
F1 score is : [0.8846079715644932, 0.8791886922276678, 0.8765366776241725, 0.8935542420095313, 0.8933308878457356, 0.8968627378405172, 0.9021640542969172, 0.8852937735407562, 0.8981691856750509, 0.912070778184549]
mean F1 score i

In [9]:
import xgboost
xgb_classifier=xgboost.XGBClassifier()

In [10]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}
xgb_grid  = RandomizedSearchCV(xgb_classifier , param_distributions = params, cv = 3, verbose = 10,scoring='accuracy')
result_xgb=xgb_grid.fit(x_new_df,y)
xgb_estimator_result = result_xgb.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=5


KeyboardInterrupt: 

In [12]:
print(xgb_estimator_result)

XGBClassifier(colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=12)


In [11]:
xgb_classifier_hyp=xgboost.XGBClassifier(colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=12)

In [12]:
scores_xgb_test = []
scores_xgb_train = []
precision_xgb =[]
recall_xgb=[]
f1_score_xgb = []
specificity_xgb =[]
gmean_xgb= []
ROC_xgb= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6,something8= get_score(xgb_classifier_hyp, X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb)
    scores_xgb_test.append(something1)
    scores_xgb_train.append(something7)
    precision_xgb.append(something2)
    recall_xgb.append(something3)
    f1_score_xgb.append(something4)
    specificity_xgb.append(something5)
    gmean_xgb.append(something6)
    ROC_xgb.append(something8)
print('Score test is :', scores_xgb_test)
print('mean Score test is :', np.mean(scores_xgb_test))
print('Score train is :', scores_xgb_train)
print('mean Score train is :', np.mean(scores_xgb_train))
print('Precision is :',precision_xgb)
print('mean Precision is :',np.mean(precision_xgb))
print('F1 score is :',f1_score_xgb)
print('mean F1 score is :',np.mean(f1_score_xgb))
print('REcall is :',recall_xgb)
print('mean Recall is :',np.mean(recall_xgb))
print('Gmean is :',gmean_xgb)
print('mean Gmean is :',np.mean(gmean_xgb))
print('ROC score is :', ROC_xgb)
print('mean ROC score is :', np.mean(ROC_xgb))

Score test is : [0.9974924774322969, 0.9964894684052157, 0.995987963891675, 0.9969909729187563, 0.9964894684052157, 0.9969909729187563, 0.995987963891675, 0.9989964877069744, 0.9964877069744105, 0.9979929754139488]
mean Score test is : 0.9969906457958924
Score train is : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
mean Score train is : 1.0
Precision is : [0.998499399759904, 0.9979016786570744, 0.9976033553025764, 0.9982003599280145, 0.9979016786570744, 0.9982003599280145, 0.9963713679282492, 0.9993983152827919, 0.9979004199160169, 0.9987987987987987]
mean Precision is : 0.9980775734158515
F1 score is : [0.9954664240271343, 0.9936375558332053, 0.9927197714431757, 0.9945531031468531, 0.9936375558332053, 0.9945531031468531, 0.9927374708624709, 0.998192952423783, 0.9936369238122995, 0.9963683740592941]
mean F1 score is : 0.9945503234588274
REcall is : [0.9924924924924925, 0.9894894894894894, 0.987987987987988, 0.990990990990991, 0.9894894894894894, 0.990990990990991, 0.9891884660096

In [13]:
from sklearn.neural_network import MLPClassifier

In [16]:
params={
 "hidden_layer_sizes"    : [2,5,10,20,50,75,100] ,
 "alpha"        : [ .001,.005,.02,.1],
 "learning_rate" : [ 'adaptive' ],

    
}
mlp_grid  = RandomizedSearchCV(MLPClassifier(max_iter=1000) , param_distributions = params, cv = 3, verbose = 10,scoring='accuracy')
result_mlp=mlp_grid.fit(x_new_df,y)
mlp_estimator_result = result_mlp.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive.
[CV 1/3; 1/10] END alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive;, score=0.919 total time= 1.6min
[CV 2/3; 1/10] START alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive.
[CV 2/3; 1/10] END alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive;, score=0.920 total time= 1.4min
[CV 3/3; 1/10] START alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive.
[CV 3/3; 1/10] END alpha=0.005, hidden_layer_sizes=75, learning_rate=adaptive;, score=0.922 total time= 1.5min
[CV 1/3; 2/10] START alpha=0.005, hidden_layer_sizes=50, learning_rate=adaptive.
[CV 1/3; 2/10] END alpha=0.005, hidden_layer_sizes=50, learning_rate=adaptive;, score=0.920 total time= 1.5min
[CV 2/3; 2/10] START alpha=0.005, hidden_layer_sizes=50, learning_rate=adaptive.
[CV 2/3; 2/10] END alpha=0.005, hidden_layer_sizes=50, learning_rate=adaptive;, score=0.91

In [10]:
print(mlp_estimator_result)

NameError: name 'mlp_estimator_result' is not defined

In [15]:
scores_mlp_test = []
scores_mlp_train = []
precision_mlp =[]
recall_mlp=[]
f1_score_mlp = []
specificity_mlp =[]
gmean_mlp= []
ROC_mlp= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 ,something6,something8= get_score(MLPClassifier(alpha=0.1, hidden_layer_sizes=50, learning_rate='adaptive',max_iter=1000),X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp)
    scores_mlp_test.append(something1)
    scores_mlp_train.append(something7)
    precision_mlp.append(something2)
    recall_mlp.append(something3)
    f1_score_mlp.append(something4)
    specificity_mlp.append(something5)
    gmean_mlp.append(something6)
    ROC_mlp.append(something8)
print('Score test is :', scores_mlp_test)
print('mean Score test is :', np.mean(scores_mlp_test))
print('Score train is :', scores_mlp_train)
print('mean Score train is :', np.mean(scores_mlp_train))
print('Precision is :',precision_mlp)
print('mean Precision is :',np.mean(precision_mlp))
print('F1 score is :',f1_score_mlp)
print('mean F1 score is :',np.mean(f1_score_mlp))
print('REcall is :',recall_mlp)
print('mean Recall is :',np.mean(recall_mlp))
print('Gmean is :',gmean_mlp)
print('mean Gmean is :',np.mean(gmean_mlp))
print('ROC score is :', ROC_mlp)
print('mean ROC score is :', np.mean(ROC_mlp))

Score test is : [0.9302908726178536, 0.9237713139418254, 0.9167502507522568, 0.9297893681043129, 0.9207622868605817, 0.9348044132397192, 0.928284854563691, 0.9282488710486704, 0.9342699448068239, 0.9377822378324134]
mean Score test is : 0.9284754413768148
Score train is : [0.9784317003845511, 0.981552694644151, 0.9823886752494009, 0.980326589756451, 0.979546341191551, 0.9829459956529009, 0.9755336342863512, 0.9828354881854658, 0.9817209094962104, 0.9780985287561302]
mean Score train is : 0.9803380557603163
Precision is : [0.8896564074497368, 0.8670734103775792, 0.8636718789637643, 0.8789523879392561, 0.8652408048518176, 0.8946826192667482, 0.9128905807766331, 0.8757889796301512, 0.8892533824493967, 0.9011928094870425]
mean Precision is : 0.8838403261192125
F1 score is : [0.8681339280404401, 0.8606538082417501, 0.8419011373427867, 0.8713370512052356, 0.8529663231780167, 0.8780964957327662, 0.8547028624568466, 0.868729690454312, 0.8788447036931337, 0.8831129273322361]
mean F1 score is : 

In [10]:
from catboost import CatBoostClassifier

In [15]:
parameters_cat = {'eta': (0.3,0.001,0.1,0.5,1,3,5),'n_estimators': (1,2,10,50,100,75,200),'reg_lambda': (0.1,1,5,3,0.5)}
cat_grid  = RandomizedSearchCV( CatBoostClassifier(), param_distributions = parameters_cat, cv = 3, verbose = 10,scoring='accuracy')
result_cat=cat_grid.fit(x_new_df,y)
cat_estimator_result = result_cat.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START eta=0.5, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4279475	total: 15.8ms	remaining: 0us
[CV 1/3; 1/10] END eta=0.5, n_estimators=1, reg_lambda=5;, score=0.833 total time=   0.3s
[CV 2/3; 1/10] START eta=0.5, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4187406	total: 22ms	remaining: 0us
[CV 2/3; 1/10] END eta=0.5, n_estimators=1, reg_lambda=5;, score=0.835 total time=   0.3s
[CV 3/3; 1/10] START eta=0.5, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4305963	total: 14.9ms	remaining: 0us
[CV 3/3; 1/10] END eta=0.5, n_estimators=1, reg_lambda=5;, score=0.833 total time=   0.3s
[CV 1/3; 2/10] START eta=0.001, n_estimators=50, reg_lambda=0.5.................
0:	learn: 0.6921624	total: 13.2ms	remaining: 649ms
1:	learn: 0.6911463	total: 24.1ms	remaining: 578ms
2:	learn: 0.6902892	total: 36.5ms	remaining: 572ms
3:	learn: 0.6894224	total: 49ms	remaining: 564m

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


[CV 1/3; 3/10] END eta=3, n_estimators=2, reg_lambda=0.5;, score=0.526 total time=   0.4s
[CV 2/3; 3/10] START eta=3, n_estimators=2, reg_lambda=0.5......................
0:	learn: 0.5873538	total: 19.7ms	remaining: 19.7ms
1:	learn: 3.2545108	total: 38.9ms	remaining: 0us


learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


[CV 2/3; 3/10] END eta=3, n_estimators=2, reg_lambda=0.5;, score=0.502 total time=   0.3s
[CV 3/3; 3/10] START eta=3, n_estimators=2, reg_lambda=0.5......................
0:	learn: 0.6144483	total: 11.3ms	remaining: 11.3ms
1:	learn: 5.3622019	total: 22.5ms	remaining: 0us


...

0:	learn: 0.9795878	total: 9.93ms	remaining: 9.93ms
1:	learn: 28.4074085	total: 21.4ms	remaining: 0us
[CV 3/3; 4/10] END eta=5, n_estimators=2, reg_lambda=0.5;, score=0.169 total time=   0.4s
[CV 1/3; 5/10] START eta=0.3, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4892297	total: 35.2ms	remaining: 0us
[CV 1/3; 5/10] END eta=0.3, n_estimators=1, reg_lambda=5;, score=0.833 total time=   0.2s
[CV 2/3; 5/10] START eta=0.3, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4819759	total: 21ms	remaining: 0us
[CV 2/3; 5/10] END eta=0.3, n_estimators=1, reg_lambda=5;, score=0.835 total time=   0.3s
[CV 3/3; 5/10] START eta=0.3, n_estimators=1, reg_lambda=5......................
0:	learn: 0.4922057	total: 10.8ms	remaining: 0us
[CV 3/3; 5/10] END eta=0.3, n_estimators=1, reg_lambda=5;, score=0.833 total time=   0.2s
[CV 1/3; 6/10] START eta=0.1, n_estimators=50, reg_lambda=0.5...................
0:	learn: 0.6054720	total: 17.5ms	remaining: 856ms
1:	learn: 0.555468

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


4:	learn: 2.2860656	total: 84ms	remaining: 1.6s
5:	learn: 10.3891184	total: 99.6ms	remaining: 1.56s
6:	learn: 7.9667571	total: 114ms	remaining: 1.51s
7:	learn: 19.8473964	total: 128ms	remaining: 1.47s
8:	learn: 12.2755508	total: 142ms	remaining: 1.43s
9:	learn: 25.0403167	total: 156ms	remaining: 1.4s
10:	learn: 15.7823800	total: 170ms	remaining: 1.37s
11:	learn: 20.3036202	total: 186ms	remaining: 1.37s
12:	learn: 14.9281306	total: 203ms	remaining: 1.36s
13:	learn: 15.0766524	total: 217ms	remaining: 1.33s
14:	learn: 14.8615612	total: 227ms	remaining: 1.29s
15:	learn: 15.4911272	total: 238ms	remaining: 1.25s
16:	learn: 16.8839442	total: 250ms	remaining: 1.22s
17:	learn: 18.7603718	total: 259ms	remaining: 1.18s
18:	learn: 16.0440068	total: 270ms	remaining: 1.15s


: 

: 

In [14]:
print(cat_estimator_result)

In [14]:
scores_cat_test = []
scores_cat_train = []
precision_cat =[]
recall_cat=[]
f1_score_cat = []
specificity_cat =[]
gmean_cat= []
ROC_cat= []


for train_index, test_index in folds.split(x_new_df,y):
    X_train_cat, X_test_cat, y_train_cat, y_test_cat = x_new_df.iloc[train_index], x_new_df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
   
    something1,something7, something2, something3, something4 , something5 , something6= get_score(CatBoostClassifier(), X_train_cat, X_test_cat, y_train_cat, y_test_cat)
    scores_cat_test.append(something1)
    scores_cat_train.append(something7)
    precision_cat.append(something2)
    recall_cat.append(something3)
    f1_score_cat.append(something4)
    specificity_cat.append(something5)
    gmean_cat.append(something6)
    #ROC_cat.append(something7)
print('Score test is :', scores_cat_test)
print('mean Score test is :', np.mean(scores_cat_test))
print('Score train is :', scores_cat_train)
print('mean Score train is :', np.mean(scores_cat_train))
print('Precision is :',precision_cat)
print('mean Precision is :',np.mean(precision_cat))
print('F1 score is :',f1_score_cat)
print('mean F1 score is :',np.mean(f1_score_cat))
print('REcall is :',recall_cat)
print('mean Recall is :',np.mean(recall_cat))
print('Gmean is :',gmean_cat)
print('mean Gmean is :',np.mean(gmean_cat))
#print('ROC score is :', ROC_cat)
#print('mean ROC score is :', np.mean(ROC_cat))

Learning rate set to 0.035347
0:	learn: 0.6600514	total: 62.9ms	remaining: 1m 2s
1:	learn: 0.6353205	total: 76.1ms	remaining: 38s
2:	learn: 0.6063626	total: 90.4ms	remaining: 30s
3:	learn: 0.5821205	total: 103ms	remaining: 25.6s
4:	learn: 0.5686502	total: 116ms	remaining: 23.1s
5:	learn: 0.5550231	total: 133ms	remaining: 22s
6:	learn: 0.5428798	total: 149ms	remaining: 21.1s
7:	learn: 0.5254768	total: 167ms	remaining: 20.7s
8:	learn: 0.5144832	total: 192ms	remaining: 21.2s
9:	learn: 0.5042306	total: 214ms	remaining: 21.2s
10:	learn: 0.4901800	total: 234ms	remaining: 21.1s
11:	learn: 0.4819962	total: 260ms	remaining: 21.4s
12:	learn: 0.4689390	total: 276ms	remaining: 20.9s
13:	learn: 0.4611788	total: 288ms	remaining: 20.3s
14:	learn: 0.4543637	total: 306ms	remaining: 20.1s
15:	learn: 0.4476745	total: 319ms	remaining: 19.6s
16:	learn: 0.4410992	total: 337ms	remaining: 19.5s
17:	learn: 0.4355563	total: 352ms	remaining: 19.2s
18:	learn: 0.4300960	total: 368ms	remaining: 19s
19:	learn: 0.425